In [53]:
import pandas as pd
from pandas.io import sql
from sqlalchemy import create_engine
from sqlalchemy.pool import Pool
import mysql.connector
import json
import csv
import os

import numpy as np
import pymysql as mysql
import matplotlib.pyplot as plt
import requests
import getpass

!pip install mysql-connector-python

In [2]:
###############################################
# Cleaning the Data
###############################################

# Extract the Data 

In [54]:
#extract the data 


go_1k = pd.DataFrame(pd.read_csv("https://github.com/RenettaNelson/ADS-507---SQL-Data-Pipeline/raw/main/GOSales_data/go_1k.csv"))
go_daily_sales = pd.DataFrame(pd.read_csv("https://github.com/RenettaNelson/ADS-507---SQL-Data-Pipeline/raw/main/GOSales_data/go_daily_sales.csv"))
go_methods = pd.DataFrame(pd.read_csv("https://github.com/RenettaNelson/ADS-507---SQL-Data-Pipeline/raw/main/GOSales_data/go_methods.csv"))
go_products = pd.DataFrame(pd.read_csv("https://github.com/RenettaNelson/ADS-507---SQL-Data-Pipeline/raw/main/GOSales_data/go_products.csv"))
go_retailers = pd.DataFrame(pd.read_csv("https://github.com/RenettaNelson/ADS-507---SQL-Data-Pipeline/raw/main/GOSales_data/go_retailers.csv"))
cpi_monthly = pd.DataFrame(pd.read_csv("https://github.com/RenettaNelson/ADS-507---SQL-Data-Pipeline/raw/main/GOSales_data/1_month_percent_change_in_CPI.csv"))
unemploy_monthly = pd.DataFrame(pd.read_csv("https://github.com/RenettaNelson/ADS-507---SQL-Data-Pipeline/raw/main/GOSales_data/Monthly_unemployment_rate.csv"))
cci_monthly = pd.DataFrame(pd.read_csv("https://raw.githubusercontent.com/RenettaNelson/ADS-507---SQL-Data-Pipeline/main/GOSales_data/CCI%20CSV.csv"))

In [55]:
go_1k.head()

,Retailer code,Product number,Date,Quantity
0,1115,125110,2016-02-09,46
1,1115,144180,2016-04-21,19
2,1115,149140,2017-02-14,11
3,1132,92110,2017-06-13,232
4,1132,96110,2018-06-20,37


In [56]:
go_daily_sales.head()

,Retailer code,Product number,Order method code,Date,Quantity,Unit price,Unit sale price
0,1201,109110,4,2015-01-12,648,76.86,71.48
1,1201,112110,4,2015-01-12,799,10.64,10.21
2,1201,115110,4,2015-01-12,755,10.71,10.28
3,1205,70240,3,2015-01-12,70,122.70,114.11
4,1205,71110,3,2015-01-12,28,95.62,92.75


In [57]:
cci_monthly.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,G-20,CCI,AMPLITUD,LTRENDIDX,M,2014-01,98.33976,NaN
1,G-20,CCI,AMPLITUD,LTRENDIDX,M,2014-02,98.64955,NaN
2,G-20,CCI,AMPLITUD,LTRENDIDX,M,2014-03,98.96025,NaN
3,G-20,CCI,AMPLITUD,LTRENDIDX,M,2014-04,99.13481,NaN
4,G-20,CCI,AMPLITUD,LTRENDIDX,M,2014-05,99.21778,NaN


In [58]:
go_products.head()

,Product number,Product line,Product type,Product,Product brand,Product color,Unit cost,Unit price
0,1110,Camping Equipment,Cooking Gear,TrailChef Water Bag,TrailChef,Clear,2.77,6.59
1,2110,Camping Equipment,Cooking Gear,TrailChef Canteen,TrailChef,Brown,6.92,12.92
2,3110,Camping Equipment,Cooking Gear,TrailChef Kitchen Kit,TrailChef,Unspecified,15.78,23.80
3,4110,Camping Equipment,Cooking Gear,TrailChef Cup,TrailChef,Silver,0.85,3.66
4,5110,Camping Equipment,Cooking Gear,TrailChef Cook Set,TrailChef,Silver,34.41,54.93


In [59]:
go_retailers.head()

,Retailer code,Retailer name,Type,Country
0,1101,ActiForme,Equipment Rental Store,France
1,1115,SportsClub,Golf Shop,France
2,1123,Anapurna,Direct Marketing,France
3,1132,Cordages Discount,Warehouse Store,France
4,1133,Altitudes extrêmes,Outdoors Shop,France


In [60]:
cpi_monthly.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,HALF1,HALF2
0,2012,0.3,0.2,0.2,0.2,-0.2,-0.1,0.0,0.6,0.5,0.3,-0.2,0.0,NaN,NaN
1,2013,0.2,0.5,-0.3,-0.2,0.0,0.2,0.2,0.2,0.0,0.1,0.2,0.3,NaN,NaN
2,2014,0.2,0.1,0.2,0.2,0.2,0.1,0.1,0.0,0.0,0.0,-0.2,-0.3,NaN,NaN
3,2015,-0.6,0.3,0.3,0.1,0.3,0.3,0.2,0.0,-0.2,0.1,0.1,-0.1,NaN,NaN
4,2016,0.0,-0.1,0.3,0.4,0.2,0.3,-0.1,0.2,0.3,0.2,0.1,0.3,NaN,NaN


In [61]:
unemploy_monthly.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,2013,8.0,7.7,7.5,7.6,7.5,7.5,7.3,7.2,7.2,7.2,6.9,6.7
1,2014,6.6,6.7,6.7,6.2,6.3,6.1,6.2,6.1,5.9,5.7,5.8,5.6
2,2015,5.7,5.5,5.4,5.4,5.6,5.3,5.2,5.1,5.0,5.0,5.1,5.0
3,2016,4.8,4.9,5.0,5.1,4.8,4.9,4.8,4.9,5.0,4.9,4.7,4.7
4,2017,4.7,4.6,4.4,4.4,4.4,4.3,4.3,4.4,4.3,4.2,4.2,4.1


# Data Transformation

In [62]:
#Modify cci table to add year and month column
cci_monthly['Year'] = pd.DatetimeIndex(cci_monthly['TIME']).year
cci_monthly['Month'] = pd.DatetimeIndex(cci_monthly['TIME']).month
cci_monthly.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes,Year,Month
0,G-20,CCI,AMPLITUD,LTRENDIDX,M,2014-01,98.33976,NaN,2014,1
1,G-20,CCI,AMPLITUD,LTRENDIDX,M,2014-02,98.64955,NaN,2014,2
2,G-20,CCI,AMPLITUD,LTRENDIDX,M,2014-03,98.96025,NaN,2014,3
3,G-20,CCI,AMPLITUD,LTRENDIDX,M,2014-04,99.13481,NaN,2014,4
4,G-20,CCI,AMPLITUD,LTRENDIDX,M,2014-05,99.21778,NaN,2014,5


In [63]:
#add year and month column to go_daily_sales
go_daily_sales['Year'] = pd.DatetimeIndex(go_daily_sales['Date']).year
go_daily_sales['Month'] = pd.DatetimeIndex(go_daily_sales['Date']).month
go_daily_sales.head()

,Retailer code,Product number,Order method code,Date,Quantity,Unit price,Unit sale price,Year,Month
0,1201,109110,4,2015-01-12,648,76.86,71.48,2015,1
1,1201,112110,4,2015-01-12,799,10.64,10.21,2015,1
2,1201,115110,4,2015-01-12,755,10.71,10.28,2015,1
3,1205,70240,3,2015-01-12,70,122.70,114.11,2015,1
4,1205,71110,3,2015-01-12,28,95.62,92.75,2015,1


In [64]:
#add year and month column to go_1k
go_1k['Year'] = pd.DatetimeIndex(go_1k['Date']).year
go_1k['Month'] = pd.DatetimeIndex(go_1k['Date']).month
go_1k.head()

,Retailer code,Product number,Date,Quantity,Year,Month
0,1115,125110,2016-02-09,46,2016,2
1,1115,144180,2016-04-21,19,2016,4
2,1115,149140,2017-02-14,11,2017,2
3,1132,92110,2017-06-13,232,2017,6
4,1132,96110,2018-06-20,37,2018,6


In [65]:
#remove spaces from column names and insert '_'
def spaces(df):
    df.columns = df.columns.str.replace(' ', '_')
    return df.head()

data = [go_1k, go_daily_sales, go_methods, go_products, go_retailers, unemploy_monthly, cpi_monthly, cci_monthly]
for i in data:
    spaces(i)

In [66]:
cci_monthly = cci_monthly.rename(columns={"TIME": "Date"})

In [49]:
### drop flag codes columns due to NAN and irrelevance
cci_monthly = cci_monthly.drop(['Flag Codes', 'LOCATION', 'INDICATOR', 'SUBJECT', 'MEASURE', 'FREQUENCY', 'Date'], axis=1)

KeyError: "['Flag Codes'] not found in axis"

In [67]:
go_1k = go_1k.drop(['Date'], axis=1)

In [68]:
#Transpose monthly unemployment data
unemployment_monthly = unemploy_monthly.T.set_axis(unemploy_monthly['Year'], axis=1, inplace=False)
unemployment_monthly = unemployment_monthly.drop('Year')
unemployment_monthly.index.names = ['Month']
unemployment_monthly = pd.DataFrame(unemployment_monthly)
unemployment_monthly

Year,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Month,,,,,,,,,,,
Jan,8.0,6.6,5.7,4.8,4.7,4.0,4.0,3.5,6.3,4.0,3.4
Feb,7.7,6.7,5.5,4.9,4.6,4.1,3.8,3.5,6.2,3.8,NaN
Mar,7.5,6.7,5.4,5.0,4.4,4.0,3.8,4.4,6.1,3.6,NaN
Apr,7.6,6.2,5.4,5.1,4.4,4.0,3.6,14.7,6.1,3.6,NaN
May,7.5,6.3,5.6,4.8,4.4,3.8,3.7,13.2,5.8,3.6,NaN
Jun,7.5,6.1,5.3,4.9,4.3,4.0,3.6,11.0,5.9,3.6,NaN
Jul,7.3,6.2,5.2,4.8,4.3,3.8,3.7,10.2,5.4,3.5,NaN
Aug,7.2,6.1,5.1,4.9,4.4,3.8,3.7,8.4,5.2,3.7,NaN
Sep,7.2,5.9,5.0,5.0,4.3,3.7,3.5,7.9,4.8,3.5,NaN


In [69]:
#Transpose monthly cpi data
cpi_monthly_t = cpi_monthly.T.set_axis(cpi_monthly['Year'], axis=1, inplace=False)
cpi_monthly_t = cpi_monthly_t.drop(['Year', 'HALF1',"HALF2"])
cpi_monthly_t.index.names = ['Month']
cpi_monthly_t = pd.DataFrame(cpi_monthly_t)
cpi_monthly_t

Year,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Month,,,,,,,,,,,
Jan,0.3,0.2,0.2,-0.6,0.0,0.4,0.4,0.0,0.2,0.2,0.6
Feb,0.2,0.5,0.1,0.3,-0.1,0.2,0.3,0.2,0.1,0.4,0.7
Mar,0.2,-0.3,0.2,0.3,0.3,0.0,0.0,0.3,-0.4,0.5,1.0
Apr,0.2,-0.2,0.2,0.1,0.4,0.1,0.3,0.4,-0.8,0.7,0.4
May,-0.2,0.0,0.2,0.3,0.2,-0.1,0.2,0.0,-0.1,0.7,0.9
Jun,-0.1,0.2,0.1,0.3,0.3,0.1,0.1,-0.1,0.4,0.8,1.2
Jul,0.0,0.2,0.1,0.2,-0.1,0.0,0.1,0.2,0.5,0.4,0.0
Aug,0.6,0.2,0.0,0.0,0.2,0.4,0.2,0.1,0.4,0.4,0.2
Sep,0.5,0.0,0.0,-0.2,0.3,0.5,0.2,0.2,0.2,0.4,0.4


In [70]:
test = pd.merge(cci_monthly, go_1k, how='inner',
                 left_on=['Month', 'Year'],
                 right_on=['Month', 'Year'])
test.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,Date,Value,Flag_Codes,Year,Month,Retailer_code,Product_number,Quantity
0,G-20,CCI,AMPLITUD,LTRENDIDX,M,2015-01,99.97893,NaN,2015,1,1133,134120,222
1,G-20,CCI,AMPLITUD,LTRENDIDX,M,2015-01,99.97893,NaN,2015,1,1137,25110,198
2,G-20,CCI,AMPLITUD,LTRENDIDX,M,2015-01,99.97893,NaN,2015,1,1137,144180,21
3,G-20,CCI,AMPLITUD,LTRENDIDX,M,2015-01,99.97893,NaN,2015,1,1192,143120,159
4,G-20,CCI,AMPLITUD,LTRENDIDX,M,2015-01,99.97893,NaN,2015,1,1194,125120,16


In [71]:
test.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,Date,Value,Flag_Codes,Year,Month,Retailer_code,Product_number,Quantity
0,G-20,CCI,AMPLITUD,LTRENDIDX,M,2015-01,99.97893,NaN,2015,1,1133,134120,222
1,G-20,CCI,AMPLITUD,LTRENDIDX,M,2015-01,99.97893,NaN,2015,1,1137,25110,198
2,G-20,CCI,AMPLITUD,LTRENDIDX,M,2015-01,99.97893,NaN,2015,1,1137,144180,21
3,G-20,CCI,AMPLITUD,LTRENDIDX,M,2015-01,99.97893,NaN,2015,1,1192,143120,159
4,G-20,CCI,AMPLITUD,LTRENDIDX,M,2015-01,99.97893,NaN,2015,1,1194,125120,16


In [72]:
test.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,Date,Value,Flag_Codes,Year,Month,Retailer_code,Product_number,Quantity
0,G-20,CCI,AMPLITUD,LTRENDIDX,M,2015-01,99.97893,NaN,2015,1,1133,134120,222
1,G-20,CCI,AMPLITUD,LTRENDIDX,M,2015-01,99.97893,NaN,2015,1,1137,25110,198
2,G-20,CCI,AMPLITUD,LTRENDIDX,M,2015-01,99.97893,NaN,2015,1,1137,144180,21
3,G-20,CCI,AMPLITUD,LTRENDIDX,M,2015-01,99.97893,NaN,2015,1,1192,143120,159
4,G-20,CCI,AMPLITUD,LTRENDIDX,M,2015-01,99.97893,NaN,2015,1,1194,125120,16


# Load transformed tables to destination database

In [73]:
# Connect to the sql server
connection = mysql.connect(host='localhost',
                         user='root',
                         password=getpass.getpass(prompt='Password:',stream=None))
                          

# create cursor
cursor = connection.cursor()

In [74]:
#Create Database
cursor.execute("CREATE DATABASE wild_wonderful_sales")

1

In [75]:
#create engine to add data to sql database "gosales"
db_data = 'mysql+mysqlconnector://' + 'root' + ':' + getpass.getpass(prompt='Password:',stream=None) + '@' + 'localhost' + ':3306/' \
       + 'wild_wonderful_sales' + '?charset=utf8mb4'
engine = create_engine(db_data)

In [79]:
def load(df, table):
    df.to_sql(table, engine, if_exists='append', index=False)

df = [go_1k, go_daily_sales, go_methods, go_products, go_retailers, unemployment_monthly, cpi_monthly_t, cci_monthly]
tables = ['go_1k', 'go_daily_sales', 'go_methods', 'go_products', 'go_retailers', 'unemployment_monthly', 'cpi_monthly_t', 'cci_monthly']

for i, j in zip(df, tables):
    load(i, j)
print('Tables loaded sucessfully :)')

Tables loaded sucessfully :)


In [80]:
#update connection to use go sales database
connection = mysql.connect(host='localhost',
                         user='root',
                         password=getpass.getpass(prompt='Password:',stream=None),
                         database='wild_wonderful_sales')

#Begin Queries

In [83]:
#Total product sales by volume
product_vol = pd.DataFrame(pd.read_sql("SELECT s.Quantity,\
                                               p.Product \
                                        FROM go_daily_sales s \
                                            JOIN go_products p \
                                            ON s.Product_number = p.Product_number \
                                        GROUP BY Product \
                                        ORDER BY Quantity DESC \
                                        LIMIT 20;", 
                                        connection))
product_vol

,Quantity,Product
0,6121,BugShield Extreme
1,4252,Granite Carabiner
2,3992,Single Edge
3,2615,BugShield Natural
4,1913,Sun Shelter 15
5,1762,BugShield Spray
6,1645,Sun Shield
7,1482,Firefly Rechargeable Battery
8,1209,BugShield Lotion
9,1154,TrailChef Kettle


In [26]:
go_daily_sales['Revenue'] = go_daily_sales['Unit_price'] * go_daily_sales['Quantity']
monthly_sales = pd.DataFrame(go_daily_sales[['Month', 'Year', 'Revenue']])
monthly_sales.head()

,Month,Year,Revenue
0,1,2015,49805.28
1,1,2015,8501.36
2,1,2015,8086.05
3,1,2015,8589.00
4,1,2015,2677.36


In [32]:
sales = pd.DataFrame(monthly_sales.groupby(['Month', 'Year'], as_index=False)['Revenue'].sum())
monthly_sales_2015 = pd.DataFrame(sales[sales.Year ==2015])
monthly_sales_2015

,Month,Year,Revenue
0,1,2015,2069245.25
2,2,2015,2359982.96
4,3,2015,1738406.68
6,4,2015,1971011.34
8,5,2015,1948362.33
10,6,2015,2322068.70
12,7,2015,2227986.32
14,8,2015,2256036.40
16,9,2015,1825648.63
17,10,2015,1789883.06


In [31]:
monthly_sales_2016 = pd.DataFrame(sales[sales.Year ==2016])
monthly_sales_2016

,Month,Year,Revenue
1,1,2016,2721919.35
3,2,2016,3006845.73
5,3,2016,2365655.31
7,4,2016,2718757.50
9,5,2016,2372444.82
11,6,2016,1789504.70
13,7,2016,2960498.77
15,8,2016,589970.59


##Create table for monthly unemployment in 2015 and 2016

##Create table for monthly consumer price index change in 2015 and 2016

In [29]:
cpi_monthly_2015_2016 = cpi_monthly.T.set_axis(cpi_monthly['Year'], axis=1, inplace=False)
cpi_monthly_2015_2016 = cpi_monthly_2015_2016.drop(['Year', 'HALF1',"HALF2"])
cpi_monthly_2015_2016.index.names = ['Month']
cpi_monthly_2015_2016 = pd.DataFrame(cpi_monthly_2015_2016[[2015, 2016]])
cpi_monthly_2015_2016

Year,2015,2016
Month,,
Jan,-0.6,0.0
Feb,0.3,-0.1
Mar,0.3,0.3
Apr,0.1,0.4
May,0.3,0.2
Jun,0.3,0.3
Jul,0.2,-0.1
Aug,0.0,0.2
Sep,-0.2,0.3


In [92]:
connection.close()